<a href="https://colab.research.google.com/github/wuabs/child-llm-assistant/blob/main/training/instruction_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import json
import torch

In [ ]:
# Загружаем дообученную модель
model_path = "./models/lora-lm"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_8bit=True, device_map="auto")
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA config
with open("lora_config.json", "r", encoding="utf-8") as f:
    config_dict = json.load(f)
lora_config = LoraConfig(**config_dict)
model = get_peft_model(model, lora_config)

In [ ]:
# Датасет
dataset = load_dataset("json", data_files="data/child_dialogs.jsonl", split="train")

def tokenize(example):
    prompt = f"### Инструкция:\\n{example['child_input']}\\n\\n### Ответ:\\n{example['friend_reply']}"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize)

In [ ]:
# Аргументы
training_args = TrainingArguments(
    output_dir="./models/lora-instruct",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=20,
    save_steps=200,
    fp16=True,
    save_total_limit=1
)

In [ ]:
# Обучение
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)
trainer.train()


In [ ]:
# Сохраняем модель
model.save_pretrained("./models/lora-instruct")
tokenizer.save_pretrained("./models/lora-instruct")